<a href="https://colab.research.google.com/github/ana-casariego/galaxy-nn-classification/blob/main/Galaxy_NN_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

By Ana Casariego

# Galaxy Classification Project

Mount to Google Drive

In [ ]:
import pandas as pd
import io

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Import the Data

In [ ]:
import pickle

# Getting back the objects:
with open('/content/gdrive/My Drive/Colab Notebooks/galaxy_data.pkl', 'rb') as f:  
    X_train, y_train, X_test = pickle.load(f)
X_test = X_test.astype('float32') / 255

Create Validation Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=24)

Establish Generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# construct the training image generator for data augmentation
gen = ImageDataGenerator(rescale = 1./255,
                         rotation_range=90, 
                         zoom_range=0.05,
                         width_shift_range=0.02, 
                         height_shift_range=0.02, 
                         shear_range=0.05,
                         horizontal_flip=True, 
                         fill_mode="nearest")

train_flow = gen.flow(X_train, y_train, batch_size=20)

gen_val = ImageDataGenerator(rescale = 1./255)
validation_generator = gen_val.flow(X_val, y_val, batch_size=20)

Import Convolutional Base

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.trainable = False

Import Previous Model (if necessary)

In [ ]:
from keras import models
from keras import layers
model = models.load_model('/content/gdrive/My Drive/Colab Notebooks/galaxy5.h5')

NameError: ignored

Create Model (if no model is being loaded)

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(22, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 512)     14336     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 146, 146, 256)     1179904   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 73, 73, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 71, 71, 256)       590080    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 69, 69, 128)       295040    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 34, 34, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 128)      

Compile Model

In [ ]:
from keras import metrics
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.RootMeanSquaredError()])

Train Model

In [ ]:
history=model.fit(train_flow, 
                  validation_data=validation_generator, 
                  epochs=30, steps_per_epoch = 100,
                  validation_steps=70)

Epoch 1/30
100/100 [==============================] - 13s 125ms/step - loss: 0.2805 - root_mean_squared_error: 0.1144 - val_loss: 0.2806 - val_root_mean_squared_error: 0.1134
Epoch 2/30
100/100 [==============================] - 13s 125ms/step - loss: 0.2753 - root_mean_squared_error: 0.1143 - val_loss: 0.2790 - val_root_mean_squared_error: 0.1148
Epoch 3/30
100/100 [==============================] - 12s 125ms/step - loss: 0.2794 - root_mean_squared_error: 0.1137 - val_loss: 0.2716 - val_root_mean_squared_error: 0.1087
Epoch 4/30
100/100 [==============================] - 13s 125ms/step - loss: 0.2818 - root_mean_squared_error: 0.1159 - val_loss: 0.2714 - val_root_mean_squared_error: 0.1109
Epoch 5/30
100/100 [==============================] - 13s 125ms/step - loss: 0.2798 - root_mean_squared_error: 0.1149 - val_loss: 0.2806 - val_root_mean_squared_error: 0.1187
Epoch 6/30
100/100 [==============================] - 12s 125ms/step - loss: 0.2821 - root_mean_squared_error: 0.1161 - val_l

Unfreeze Part of Convolutional Base

In [ ]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

Re-Compile and Train Model

In [ ]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=[metrics.RootMeanSquaredError()])

history=model.fit(train_flow, 
                  validation_data=validation_generator, 
                  epochs=40, 
                  steps_per_epoch=10, 
                  validation_steps=20)

Epoch 1/40
10/10 [==============================] - 4s 262ms/step - loss: 0.2674 - root_mean_squared_error: 0.1148 - val_loss: 0.2774 - val_root_mean_squared_error: 0.1135
Epoch 2/40
10/10 [==============================] - 2s 241ms/step - loss: 0.2845 - root_mean_squared_error: 0.1171 - val_loss: 0.2805 - val_root_mean_squared_error: 0.1134
Epoch 3/40
10/10 [==============================] - 2s 241ms/step - loss: 0.2779 - root_mean_squared_error: 0.1141 - val_loss: 0.2800 - val_root_mean_squared_error: 0.1118
Epoch 4/40
10/10 [==============================] - 2s 241ms/step - loss: 0.2833 - root_mean_squared_error: 0.1149 - val_loss: 0.2837 - val_root_mean_squared_error: 0.1177
Epoch 5/40
10/10 [==============================] - 2s 241ms/step - loss: 0.2753 - root_mean_squared_error: 0.1045 - val_loss: 0.2698 - val_root_mean_squared_error: 0.1087
Epoch 6/40
10/10 [==============================] - 2s 241ms/step - loss: 0.2727 - root_mean_squared_error: 0.1125 - val_loss: 0.2715 - val_

Evaluate Average RMSE, Maximum, and Minimum Values

In [ ]:
import statistics

statistics.mean(history.history['val_root_mean_squared_error'])

0.11368384137749672

In [ ]:
max(history.history['val_root_mean_squared_error'])

0.11827083677053452

In [ ]:
min(history.history['val_root_mean_squared_error'])

0.10871231555938721

Save Model

In [ ]:
model.save('/content/gdrive/My Drive/Colab Notebooks/galaxy.h5')

Export to CSV

In [ ]:
import pandas as pd
import numpy as np
pred = model.predict(X_test)
out = pd.DataFrame(pred)
out.to_csv("/content/gdrive/My Drive/Colab Notebooks/galaxybest.csv")